# **Initialization** 

In [1]:
import pandas as pd
import torch

### Control

In [2]:
# Paths
RAW_DATA_FOLDER = 'raw_data'
TARGET_FILE_PATH = 'unprocessed_data'

# Flow Controls
RELOAD_RAW_DATA = False
DATA = 'depression'  # Options: 'depression', 'insomnia', 'electrical_circuit'

# System variables
STATE = 42
TEST_SET_FRACTION = 0.20
MISSING_VALUES_THRESHOLD = 0.50
SAMPLES_ELECTRICAL_CIRCUIT = 5000
VERBOSE = True

# **Data Preparation**

### Merge raw data files

In [3]:
from raw_data_loader import load_raw_data

if (RELOAD_RAW_DATA):
    load_raw_data(RAW_DATA_FOLDER, TARGET_FILE_PATH)

### Preprocessing and Split

In [ ]:
from preprocessing_depression import clean_and_preprocess_depression_data
from preprocessing_insomnia import clean_and_preprocess_insomnia_data
from preprocessing_electrical_circuit import gen_and_preprocess_ec_data

dataset = pd.read_csv(TARGET_FILE_PATH + '/depression_data.csv')

if DATA == 'depression':
    X_train, X_test, y_train, y_test, y_embed_train, y_embed_test = clean_and_preprocess_depression_data(dataset, RAW_DATA_FOLDER, TEST_SET_FRACTION, STATE, MISSING_VALUES_THRESHOLD)
elif DATA == 'insomnia':
    X_train, X_test, y_train, y_test, y_embed_train, y_embed_test = clean_and_preprocess_insomnia_data(dataset, RAW_DATA_FOLDER, TEST_SET_FRACTION, STATE, MISSING_VALUES_THRESHOLD)
elif DATA == 'electrical_circuit':
    ec_X_train, ec_X_test, ec_y_train, ec_y_test, ec_y_embed_train, ec_y_embed_test = gen_and_preprocess_ec_data(SAMPLES_ELECTRICAL_CIRCUIT, TEST_SET_FRACTION, STATE)
else:
    raise ValueError("Invalid dataset selected")

#TODO Fix issue with the time columns SLQ300/310/320/330 in depression and processing

Dropped 142 columns with >50.0% missing values
Shape after dropping high-missing columns: (3333, 109)
Replaced 708 special code values with NaN
Replaced 186 special code values with NaN
Ordinal columns: 27
Nominal columns: 3
Binary columns: 2
Numerical columns: 74
Object columns (excluded): 3
Total columns identified: 109


c:\Users\jaspe\Documents\Github_projects\DSAIT4020-A2\venv\Lib\site-packages\sklearn\impute\_base.py:641: UserWarning: Skipping features without any observed values: ['SLQ300' 'SLQ310' 'SLQ320' 'SLQ330']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\jaspe\Documents\Github_projects\DSAIT4020-A2\venv\Lib\site-packages\sklearn\impute\_base.py:641: UserWarning: Skipping features without any observed values: ['SLQ300' 'SLQ310' 'SLQ320' 'SLQ330']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\jaspe\Documents\Github_projects\DSAIT4020-A2\venv\Lib\site-packages\sklearn\impute\_base.py:641: UserWarning: Skipping features without any observed values: ['SLQ300' 'SLQ310' 'SLQ320' 'SLQ330']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


# **Models**

In [5]:
from baseline_models import train_multitarget_baseline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

### Training Bayesian Models

In [6]:
nb_model = GaussianNB()
y_pred_nb, acc_nb = train_multitarget_baseline(
                            model=nb_model,
                            is_classifier=False,
                            X_train=X_train,
                            X_test=X_test,
                            y_embed_train=y_embed_train,
                            y_embed_test=y_embed_test,
                            verbose=VERBOSE)



 ######################################## GaussianNB Multitarget Regressor ########################################
Train MSE per embedding: [25.79027903 10.5379538   1.81578158  1.41134113  1.6249625   3.31743174
  2.22232223 12.61056106  7.51755176]
Test MSE per embedding: [26.91007194 10.45683453  2.0263789   1.47601918  1.68465228  3.55275779
  2.40167866 13.19064748  7.89208633]
Average train MSE: 7.427576090942428
Average test MSE: 7.7323474553690374


### Training Random Forests

In [7]:
# Parameter
N_ESTIMATORS = 100

In [8]:
rf_model = RandomForestRegressor(n_estimators=N_ESTIMATORS, random_state=STATE, n_jobs=-1)
y_pred_rf, mse_rf = train_multitarget_baseline(
                                    model=rf_model, 
                                    is_classifier=False, 
                                    X_train=X_train, 
                                    X_test=X_test, 
                                    y_embed_train=y_embed_train, 
                                    y_embed_test=y_embed_test,
                                    verbose=VERBOSE)



 ######################################## RandomForestRegressor Multitarget Regressor ########################################
Train MSE per embedding: [0.13215092 0.0942333  0.12918404 0.11008542 0.11006673 0.08917942
 0.07950783 0.07936454 0.04251734]
Test MSE per embedding: [0.67283453 0.5841699  0.82962218 0.70119448 0.7444193  0.65046775
 0.6631548  0.50801535 0.25973645]
Average train MSE: 0.09625439210587726
Average test MSE: 0.623734972022382


### Training Logistic Models

In [9]:
# Parameters
MAX_ITERATIONS = 1000

In [10]:
log_model = LogisticRegression(max_iter=MAX_ITERATIONS, class_weight='balanced', random_state=STATE)
y_pred_log, acc_log = train_multitarget_baseline(
                            model=log_model,
                            is_classifier=True,
                            X_train=X_train,
                            X_test=X_test,
                            y_embed_train=y_embed_train,
                            y_embed_test=y_embed_test,
                            verbose=VERBOSE)



 ######################################## LogisticRegression Multitarget Classifier ########################################
Train accuracy per embedding: [0.53045305 0.56675668 0.43114311 0.43054305 0.49654965 0.55745575
 0.5529553  0.62676268 0.70507051]
Test accuracy per embedding: [0.44604317 0.51798561 0.35251799 0.382494   0.43645084 0.5323741
 0.49280576 0.57913669 0.66906475]
Average train accuracy: 0.5441877521085442
Average test accuracy: 0.4898747668531841


### Train Split MLP

In [11]:
from split_model import train_split_model

torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_split_model( X_train, X_test, y_train, y_test, y_embed_train, y_embed_test )

TypeError: 'int' object is not callable

### Train Joint MLP

In [ ]:
from joint_model import train_joint_model

train_joint_model( X_train, X_test, y_train, y_test, y_embed_train, y_embed_test )